In [1]:
import requests
import time
from typing import Optional, Tuple, Dict, List
from googlemaps import Client

In [3]:
def test_nominatim_geocoding(address: str) -> Optional[Dict]:
    """Test geocoding with Nominatim"""
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': address,
        'format': 'json',
        'limit': 1
    }
    headers = {
        'User-Agent': 'Wisconsin Supper Clubs Mapping Project Test'
    }
    
    try:
        time.sleep(1)  # Respect Nominatim's rate limit
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        
        results = response.json()
        if results:
            return {
                'coordinates': (float(results[0]['lon']), float(results[0]['lat'])),
                'display_name': results[0]['display_name'],
                'type': results[0].get('type'),
                'class': results[0].get('class')
            }
        return None
        
    except Exception as e:
        print(f"Nominatim error: {str(e)}")
        return None

def test_google_geocoding(address: str, api_key: str) -> Optional[Dict]:
    """Test geocoding with Google Maps API"""
    gmaps = Client(key=api_key)
    
    try:
        result = gmaps.geocode(address)
        
        if result and len(result) > 0:
            location = result[0]['geometry']['location']
            return {
                'coordinates': (location['lng'], location['lat']),
                'formatted_address': result[0]['formatted_address'],
                'place_id': result[0]['place_id'],
                'types': result[0]['types'],
                'address_components': result[0]['address_components']
            }
        return None
        
    except Exception as e:
        print(f"Google Maps error: {str(e)}")
        return None

def compare_geocoding_results(address: str, api_key: str):
    """Compare results from both geocoding services"""
    print(f"\nTesting address: {address}")
    print("-" * 50) 
    # Test Nominatim
    print("\n1. Testing Nominatim:")
    nominatim_result = test_nominatim_geocoding(address)
    
    if nominatim_result:
        print("✓ Success!")
        print(f"Coordinates: {nominatim_result['coordinates']}")
        print(f"Display Name: {nominatim_result['display_name']}")
        print(f"Type: {nominatim_result['type']}")
        print(f"Class: {nominatim_result['class']}")
    else:
        print("✗ Failed to geocode with Nominatim")
    
    # Test Google Maps
    print("\n2. Testing Google Maps:")
    google_result = test_google_geocoding(address, api_key)
    
    if google_result:
        print("✓ Success!")
        print(f"Coordinates: {google_result['coordinates']}")
        print(f"Formatted Address: {google_result['formatted_address']}")
        print(f"Place ID: {google_result['place_id']}")
        print(f"Types: {google_result['types']}")
        print("\nAddress Components:")
        for component in google_result['address_components']:
            print(f"  - {component['long_name']} ({', '.join(component['types'])})")
    else:
        print("✗ Failed to geocode with Google Maps")
    
    # Compare results if both succeeded
    if nominatim_result and google_result:
        print("\n3. Comparison:")
        nom_coords = nominatim_result['coordinates']
        google_coords = google_result['coordinates']
        
        # Calculate rough distance between points (this is a simplified calculation)
        from math import sqrt
        diff = sqrt((nom_coords[0] - google_coords[0])**2 + 
                   (nom_coords[1] - google_coords[1])**2)
        
        print(f"Coordinate difference: {diff:.6f} degrees")
        print(f"Nominatim display name: {nominatim_result['display_name']}")
        print(f"Google display name: {google_result['formatted_address']}")

In [4]:
# Replace with your Google Maps API key
GOOGLE_MAPS_API_KEY = 'YOUR_GOOGLE_MAPS_API_KEY'

# Test address
test_address = "1380 Friess Lake Road Hubertus, WI"

# Run comparison
compare_geocoding_results(test_address, GOOGLE_MAPS_API_KEY)


Testing address: 1380 Friess Lake Road Hubertus, WI
--------------------------------------------------

1. Testing Nominatim:
✗ Failed to geocode with Nominatim

2. Testing Google Maps:
✓ Success!
Coordinates: (-88.2877987, 43.2389669)
Formatted Address: 1380 Friess Lake Rd, Hubertus, WI 53033, USA
Place ID: ChIJwXubeypSBIgRK15Ng8bo2sw
Types: ['premise']

Address Components:
  - 1380 (street_number)
  - Friess Lake Road (route)
  - Hubertus (locality, political)
  - Richfield (administrative_area_level_3, political)
  - Washington County (administrative_area_level_2, political)
  - Wisconsin (administrative_area_level_1, political)
  - United States (country, political)
  - 53033 (postal_code)
  - 9751 (postal_code_suffix)
